1. dx, dy
2. next nodes:
   1. pairwise 要 python 3.10
   2. 注意coding里面的xy与数学里面的xy正好相反
      1. up: (-1, 0), down: (1, 0), left: (0, -1), right: (0, 1)
   3. 如果使用dx dy 注意要设 nx = x+dx, ny = y + dy 不要直接在xy上面修改
      1. 这一点在maze的题目里面很容易错 因为mazeI, II, III都是走到底 在while loop里面容易 x += dx, y += dy
      2. 使用lambda就没有所谓
3. 对比DFS，BFS的好处是碰到hole可以考虑直接return，因为此时一定steps最少。唯一需要注意的是要考虑lexical order。我们用heap来pop node就可以控制lexical order了

In [1]:
# my draft
from typing import *
from itertools import pairwise
def findShortestWay(maze: List[List[int]], ball: List[int], hole: List[int]) -> str:

    # dfs
    G = maze
    start = ball
    m, n = len(G), len(G[0])
    sx, sy = start
    ex, ey = hole

    paths = []
    steps = 0
    dirs = {}
    for (dx, dy), c in zip(pairwise((-1, 0, 1, 0, -1)), "urdl"): # pairwise('ABCDEFG') --> AB BC CD DE EF FG
        dirs[(dx, dy)] = c

In [3]:
dirs = {}
for (dx, dy), c in zip(pairwise((-1, 0, 1, 0, -1)), "urdl"): # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    dirs[(dx, dy)] = c
dirs

{(-1, 0): 'u', (0, 1): 'r', (1, 0): 'd', (0, -1): 'l'}

In [ ]:
# Maze I 使用DFS 一定要有vis(code中是stopped) https://leetcode.com/problems/the-maze/solutions/150523/python-elegant-dfs-solution/
class Solution:
    def hasPath(self, maze, start, destination):
        m, n, stopped = len(maze), len(maze[0]), set()
        def dfs(x, y):
            if (x, y) in stopped: # 1. DFS vis 控制什么时候停止搜索。这个很重要 因为dfs理论上来说是可以重复step on任何一个grid的
                return False
            stopped.add((x, y))
            if [x, y] == destination: # 2.常规的ending state condition
                return True
            for i, j in [(-1, 0) , (1, 0), (0, -1), (0, 1)]:
                newX, newY = x, y # easy fail：注意是new x new y 每一个direction都要从original的xy开始 所以不能直接覆盖修改 xy
                while 0 <= newX + i < m and 0 <= newY + j < n and maze[newX + i][newY + j] != 1: # 注意墙体的限制
                    newX += i
                    newY += j
                if dfs(newX, newY):
                    return True
            return False
        return dfs(*start)

"""
next node 方式: step by step
这就是经典的BFS了，并且对path进行了存储。
唯一的问题是，因为题目要求返回的path不但要step最小，还要是lexical order最优的。
而BFS只能保证step最小。
于是该用优先队列来存储nex nodes。按照（step instruct，x, y）排序
这样当到达hole的时候一定是最优的路径

"""

"""
next node 方式: 走到可以停下来为止 或者 掉到hole里面
因为next node的选择方式，这里其实不算是经典的BFS
BFS是进行step by step的遍历，每个node（state node）只会visit一次，第一次arrive at target必定是最短距离
但在当前xy时，每个next node的步长都可能不一样，这就不是step by step的了。
反而借用了DFS和recursion求最值的方式，创建一个distance矩阵，如果当前状态的step比distance里面存储的还要大 则放弃当前的node
这样最后得到的distance里面hole的step值一定是最小的(到达hole算法并不会终止 distance矩阵的hole step也会被多次update)
distance这个矩阵起到了dfs里面的memo和bfs里面的visited的作用
"""

In [ ]:
"""
next node 方式: step by step
这就是经典的BFS了，并且对path进行了存储。
唯一的问题是，因为题目要求返回的path不但要step最小，还要是lexical order最优的。
而BFS只能保证step最小。
于是该用优先队列来存储nex nodes。按照（step instruct，x, y）排序
这样当到达hole的时候一定是最优的路径

"""
import heapq
class Solution:
    # https://leetcode.com/problems/the-maze-iii/solutions/150550/python-short-priorityqueue-solution-beats-100/
    def findShortestWay(self, maze, ball, hole):
        m, n, q, stopped = len(maze), len(maze[0]), [(0, "", ball[0], ball[1])], {(ball[0], ball[1]): [0, ""]}
        while q:
            dist, pattern, x, y = heapq.heappop(q)
            if [x, y] == hole:
                return pattern
            for i, j, p in ((-1, 0, "u"), (1, 0, "d"), (0, -1, "l"), (0, 1, "r")):
                newX, newY, d = x, y, 0
                while 0 <= newX + i < m and 0 <= newY + j < n and maze[newX + i][newY + j] != 1:
                    newX += i
                    newY += j
                    d += 1
                    if [newX, newY] == hole:
                        break
                if (newX, newY) not in stopped or [dist + d, pattern + p] < stopped[(newX, newY)]:
                    stopped[(newX, newY)] = [dist + d, pattern + p]
                    heapq.heappush(q, (dist + d, pattern + p, newX, newY))
        return "impossible"
    
class Solution:
    # https://leetcode.com/problems/the-maze-iii/solutions/3317771/python-bfs-with-heap-clean-code-easy-to-understand-with-explanations-and-comments/
    def findShortestWay(self, maze: List[List[int]], ball: List[int], hole: List[int]) -> str:
        directions = {
            'u': (-1, 0),
            'd': (1, 0),
            'l': (0, -1),
            'r': (0, 1)
        }
        m = len(maze)
        n = len(maze[0])

        visited = set()
        queue = []
        # obj in queue: (distance, instructions, i, j)
        heapq.heappush(queue, (0, '', ball[0], ball[1]))
        # obj in visited set: (last instruction, i, j)
        visited.add(('', ball[0], ball[1]))

        while queue:
            dist, inst, i, j = heapq.heappop(queue)
            if (i, j) == (hole[0], hole[1]):
                return inst

            if inst:
                # see if the ball is still moving from last instruction
                di, dj = directions[inst[-1]]
                ni, nj = i + di, j + dj
                if (0 <= ni < m) and (0 <= nj < n) and maze[ni][nj] == 0:
                    # we have to continue moving in current direction
                    heapq.heappush(queue, (dist + 1, inst, ni, nj))
                    continue

            for direction in directions:
                di, dj = directions[direction]
                ni, nj = i + di, j + dj
                if not ((0 <= ni < m) and (0 <= nj < n)):
                    continue
                if maze[ni][nj] == 1:
                    continue
                if (direction, ni, nj) in visited:
                    # no need to explore again if we have already visited the same point on the same moving direction
                    continue
                heapq.heappush(queue, (dist + 1, inst + direction, ni, nj))
                visited.add((direction, ni, nj))

        return 'impossible'

In [ ]:
"""
next node 方式: 走到可以停下来为止 或者 掉到hole里面
因为next node的选择方式，这里其实不算是经典的BFS
BFS是进行step by step的遍历，每个node（state node）只会visit一次，第一次arrive at target必定是最短距离
但在当前xy时，每个next node的步长都可能不一样，这就不是step by step的了。
反而借用了DFS和recursion求最值的方式，创建一个distance矩阵，如果当前状态的step比distance里面存储的还要大 则放弃当前的node
这样最后得到的distance里面hole的step值一定是最小的(到达hole算法并不会终止 distance矩阵的hole step也会被多次update)
distance这个矩阵起到了dfs里面的memo和bfs里面的visited的作用
"""
class Solution:
    def findShortestWay(self, maze: List[List[int]], ball: List[int], hole: List[int]) -> str:
        Row, Col = len(maze), len(maze[0])
        dist = [[float('inf') for _ in range(Col)] for _ in range(Row)]
        path = [['impossible' for _ in range(Col)] for _ in range(Row)]

        dist[ball[0]][ball[1]] = 0
        path[ball[0]][ball[1]] = ""

        Q = [(ball[0], ball[1])]
        while Q:
            r,c = Q.pop(0)
            for dr,dc,direction in ((0,1,'r'), (1,0,'d'), (0,-1,'l'), (-1,0,'u')):
                nr = r + dr
                nc = c + dc
                acc_step = dist[r][c] + 1
                new_path = path[r][c] + direction
                while 0 <= nr < Row and 0 <= nc < Col and maze[nr][nc] == 0 and not(nr-dr==hole[0] and nc-dc==hole[1]):
                        #不能是从洞滚过来的
                    nr += dr        #沿着这个方向继续前进
                    nc += dc
                    acc_step += 1
                nr -= dr            #碰壁了，回退一步
                nc -= dc
                acc_step -= 1
                if acc_step < dist[nr][nc] or (acc_step == dist[nr][nc] and new_path < path[nr][nc]):   #更短，或者字典序更小
                    dist[nr][nc] = acc_step
                    path[nr][nc] = new_path
                    if not (nr == hole[0] and nc == hole[1]):   #进了洞，就不可能再滚动了
                        Q.append( (nr,nc) )
        
        return path[hole[0]][hole[1]]

"""作者：HanXin
链接：https://leetcode.cn/problems/the-maze-iii/solutions/659045/c-python3-bfs-zhu-yi-jin-dong-liao-jiu-b-ozpb/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。"""
class Solution:
    def findShortestWay(self, maze: List[List[int]], ball: List[int], hole: List[int]) -> str:
                                                    
        directions = [(-1,0,'u'),(0,1,'r'),(0,-1,'l'),(1,0,'d')]    
                                                    # 定义上下左右四个方向和对应字符
        m = len(maze)                               # 获取矩阵大小
        n = len(maze[0])
        queue = [(ball[0],ball[1])]                 # 构造队列，并将起始位置包含其中
                                                    # distance 保存从起点到每个点的距离
                                                    # string 保存每个点对应的字符串
        distance = [[float('inf')]*n for _ in range(m)]
        string = [["impossible"]*n for _ in range(m)]
        distance[ball[0]][ball[1]] = 0              # 对起点的distance和string进行初始化
        string[ball[0]][ball[1]] = ""

        while queue:
            i,j = queue.pop(0)                      # 弹出坐标值i,j
            
            for dx,dy,letter in directions:         # 对四个方向进行遍历，letter保存了操作对应的字符
                x,y,step,word =i+dx,j+dy,distance[i][j],string[i][j]
                while 0<=x<m and 0<=y<n and maze[x][y] == 0 and (x-dx!=hole[0] or y-dy!=hole[1]):                                      
                                                    # 当x,y坐标合法，并且对应值为0
                                                    # 且没有越过hole时
                    x = x+dx                        # 继续前进，模拟小球的滚动过程
                    y = y+dy                        
                    step += 1                       # 记录步数

                x = x - dx
                y = y - dy
                # 有可能返回的是hole

                if distance[x][y] > step or (distance[x][y]==step and word+letter<string[x][y]):           
                                                    # 如果起点到该点的距离比当前距离大
                                                    # 或者相等，但是字符串的字典序大
                                                    # 更新该距离和字符串，并将坐标加入队列
                    distance[x][y] = step
                    string[x][y] = word+letter
                    #print(x,y,string[x][y])   
                    if x!=hole[0] or y!=hole[1]:    # 当坐标不是hole坐标时
                        queue.append((x,y))         # 将其添加到队列中
                                           
        return string[hole[0]][hole[1]]
"""
作者：heimisa000
链接：https://leetcode.cn/problems/the-maze-iii/solutions/74181/python3-bfsyan-du-you-xian-sou-suo-shuang-100da-li/
来源：力扣（LeetCode）
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。"""